In [2]:
import h5py
import numpy as np
import pandas as pd
import json
import yaml
from ludwig import LudwigModel
import copy
import ray
from ludwig.utils.misc import merge_dict
import re

In [3]:
pattern = "^{{.*}}$"
list_index_pattern = "^\[\d+\]$"


def get_keys_from_list(l, path=""):
    parameters = []

    for index, v in enumerate(l):
        if isinstance(l[index], str):
             if re.match(pattern, l[index], flags=0) is not None:
                parameters.append([path + "[" + index + "]", v])

        elif isinstance(l[index], dict):
            p = get_keys_from_dict(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)
        elif isinstance(l[index], list):
            p = get_keys_from_list(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)

    return parameters


def get_keys_from_dict(dct, path=""):
    parameters = []
    for k, v in dct.items():
        if isinstance(dct[k], str):
            if re.match(pattern, dct[k], flags=0) is not None:
                parameters.append([path + k,dct[k]])

        elif isinstance(dct[k], dict):
            p = get_keys_from_dict(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)

        elif isinstance(dct[k], list):
            p = get_keys_from_list(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)

    return parameters



def update_param(dct, path, value):
    if len(path) == 1:
        dct[path[0]] = value
    else:
        l = re.match(list_index_pattern, path[0], flags=0)
        if l is not None and isinstance(dct, list):
            index = int(l.group(0)[1:-1])
            update_param(dct[index], path[1:], value)
        else:
             update_param(dct[path[0]], path[1:], value)


def build_model(base_model, config):
    for k, v in config.items():
        p = k.split('->')
        update_param(base_model, p, v)
    return base_model

In [8]:
from ray.tune import grid_search

def build_search_space(annotated_model):
    pattern = "^{{(.*)}}$"
    grid_search_space = {}
    keys = get_keys_from_dict(annotated_model)
    for a in keys:
        grid_search_space[a[0]]= grid_search(yaml.load(re.match(pattern, a[1], flags=0)[1]))

    return grid_search_space


In [11]:
data = '/Users/bm255022/Projects/Teradata/ludwig/examples/hyperparameters/titanic.hdf5'
metadata = '/Users/bm255022/Projects/Teradata/ludwig/examples/hyperparameters/titanic.json'

def train(base, config, reporter):
    new_model_def = build_model(base, config)
    model = LudwigModel(new_model_def)
    train_stats = model.train(data_hdf5=data, train_set_metadata_json=metadata)
    return reporter(mean_accuracy=train_stats['validation']['Survived']['accuracy'][-1], done=True)


In [10]:
from ray.tune import register_trainable, grid_search, run_experiments

ray.shutdown()
ray.init()

with open("titanic_full_h.yaml", 'r') as stream:
    annotated_model = yaml.load(stream)

grid_search_space = build_search_space(annotated_model)
register_trainable('train', lambda cfg, rptr: train(annotated_model, cfg, rptr))
run_experiments({'my_experiment': {
    'run': 'train',
    'stop': {'mean_accuracy': 0.9},
    'config': grid_search_space}
    })


2019-03-25 11:46:06,441	WARNING worker.py:1381 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-03-25 11:46:06,443	INFO node.py:439 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-25_11-46-06_63973/logs.
2019-03-25 11:46:06,558	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:42174 to respond...
2019-03-25 11:46:06,684	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:33709 to respond...
2019-03-25 11:46:06,689	INFO services.py:761 -- Starting Redis shard with 3.44 GB max memory.
2019-03-25 11:46:06,719	INFO services.py:1449 -- Starting the Plasma object store with 5.15 GB memory using /tmp.
/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please re

{'combiner': {'fc_size': '{{[24,48,64]}}', 'num_fc_layers': 1, 'type': 'concat'}, 'input_features': [{'name': 'Pclass', 'type': 'category'}, {'name': 'Sex', 'type': 'category'}, {'name': 'Age', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'SibSp', 'type': 'numerical'}, {'name': 'Parch', 'type': 'numerical'}, {'name': 'Fare', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'Embarked', 'representation': '{{[sparse, dense]}}', 'type': 'category'}], 'output_features': [{'name': 'Survived', 'type': 'binary'}], 'training': {'batch_size': '{{[8,16,32,64]}}', 'bucketing_field': None, 'decay': False, 'decay_rate': 0.96, 'decay_steps': 10000, 'dropout_rate': 0.0, 'early_stop': 5, 'epochs': 100, 'gradient_clipping': None, 'increase_batch_size_on_plateau': 0, 'increase_batch_size_on_plateau_max': 512, 'increase_batch_size_on_plateau_patience': 5, 'increase_batch_size_on_plateau_rate': 2, 'learning_rate': 0.001, 'learning_rate_warmup_epochs':

(pid=64736) ******************
(pid=64736) {'combiner': {'fc_size': 48, 'num_fc_layers': 1, 'type': 'concat'}, 'input_features': [{'name': 'Pclass', 'type': 'category'}, {'name': 'Sex', 'type': 'category'}, {'name': 'Age', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'SibSp', 'type': 'numerical'}, {'name': 'Parch', 'type': 'numerical'}, {'name': 'Fare', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'Embarked', 'representation': 'dense', 'type': 'category'}], 'output_features': [{'name': 'Survived', 'type': 'binary'}], 'training': {'batch_size': 8, 'bucketing_field': None, 'decay': False, 'decay_rate': 0.96, 'decay_steps': 10000, 'dropout_rate': 0.0, 'early_stop': 5, 'epochs': 100, 'gradient_clipping': None, 'increase_batch_size_on_plateau': 0, 'increase_batch_size_on_plateau_max': 512, 'increase_batch_size_on_plateau_patience': 5, 'increase_batch_size_on_plateau_rate': 2, 'learning_rate': 0.001, 'learning_rate_warmup_epochs': 5

(pid=64733) ******************
(pid=64733) {'combiner': {'fc_size': 48, 'num_fc_layers': 1, 'type': 'concat'}, 'input_features': [{'name': 'Pclass', 'type': 'category'}, {'name': 'Sex', 'type': 'category'}, {'name': 'Age', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'SibSp', 'type': 'numerical'}, {'name': 'Parch', 'type': 'numerical'}, {'name': 'Fare', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'Embarked', 'representation': 'sparse', 'type': 'category'}], 'output_features': [{'name': 'Survived', 'type': 'binary'}], 'training': {'batch_size': 16, 'bucketing_field': None, 'decay': False, 'decay_rate': 0.96, 'decay_steps': 10000, 'dropout_rate': 0.0, 'early_stop': 5, 'epochs': 100, 'gradient_clipping': None, 'increase_batch_size_on_plateau': 0, 'increase_batch_size_on_plateau_max': 512, 'increase_batch_size_on_plateau_patience': 5, 'increase_batch_size_on_plateau_rate': 2, 'learning_rate': 0.001, 'learning_rate_warmup_epochs':

Result for train_1_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8:
  date: 2019-03-25_11-46-45
  done: true
  experiment_id: 8db1f1b567484fe2b0487f064cca5756
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.7654320987654321
  node_ip: 10.0.9.192
  pid: 64732
  time_since_restore: 12.087407112121582
  time_this_iter_s: 12.087407112121582
  time_total_s: 12.087407112121582
  timestamp: 1553528805
  timesteps_since_restore: 0
  training_iteration: 1
  
(pid=64732) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=64741) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
(pid=64741)   return f(*args, **kwds)
Result for train_3_combi

...

(pid=64867) ******************
(pid=64867) {'combiner': {'fc_size': 24, 'num_fc_layers': 1, 'type': 'concat'}, 'input_features': [{'name': 'Pclass', 'type': 'category'}, {'name': 'Sex', 'type': 'category'}, {'name': 'Age', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'SibSp', 'type': 'numerical'}, {'name': 'Parch', 'type': 'numerical'}, {'name': 'Fare', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'Embarked', 'representation': 'sparse', 'type': 'category'}], 'output_features': [{'name': 'Survived', 'type': 'binary'}], 'training': {'batch_size': 64, 'bucketing_field': None, 'decay': False, 'decay_rate': 0.96, 'decay_steps': 10000, 'dropout_rate': 0.0, 'early_stop': 5, 'epochs': 100, 'gradient_clipping': None, 'increase_batch_size_on_plateau': 0, 'increase_batch_size_on_plateau_max': 512, 'increase_batch_size_on_plateau_patience': 5, 'increase_batch_size_on_plateau_rate': 2, 'learning_rate': 0.001, 'learning_rate_warmup_epochs':

(pid=64875) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
(pid=64875)   "Use dataset[()] instead.", H5pyDeprecationWarning)
(pid=64864) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=64875) 2019-03-25 11:47:47.386003: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Result for train_19_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=64:
  date: 2019-03-25_11-47-49
  done: true
  experiment_id: 2fe004f1ff6247ea9df47a3534b8160e
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.8148148148148148
  node_ip: 10.0.9.192
  pid: 64870
  time_since_restore: 13.062063217163086


(pid=64867) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
Result for train_22_combiner->fc_size=48,input_features->[6]->representation=dense,training->batch_size=64:
  date: 2019-03-25_11-47-59
  done: true
  experiment_id: 4fbc454ac4594273846fb577a93986a8
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.7901234567901234
  node_ip: 10.0.9.192
  pid: 64875
  time_since_restore: 13.09470796585083
  time_this_iter_s: 13.09470796585083
  time_total_s: 13.09470796585083
  timestamp: 1553528879
  timesteps_since_restore: 0
  training_iteration: 1
  
Result for train_21_combiner->fc_size=24,input_features->[6]->representation=dense,training->batch_size=64:
  date: 2019-03-25_11-47-59
  done: true
  experiment_id: 48bb69cff83e4482b144edffdea1994d
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.7901234567901234
  no

[train_0_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=8,
 train_1_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8,
 train_2_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=8,
 train_3_combiner->fc_size=24,input_features->[6]->representation=dense,training->batch_size=8,
 train_4_combiner->fc_size=48,input_features->[6]->representation=dense,training->batch_size=8,
 train_5_combiner->fc_size=64,input_features->[6]->representation=dense,training->batch_size=8,
 train_6_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=16,
 train_7_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=16,
 train_8_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=16,
 train_9_combiner->fc_size=24,input_features->[6]->representation=dense,training->batch_size=16,
 train_10_combiner->fc_size=48

(pid=64865) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
